## Create lists for days and behaviours and a tuple for every (day, behaviour) combination

In [480]:
#create pairs of days and behaviours
list_day_no = range(1, 9) + range(11, 31)
list_behaviours = ['SW', 'FO','AR', 'BLM', 'BNV', 'BR', 'DF', 'DF-TFW', 'DR', 'DS', 'DS-TFW', 'DU', 'GF', 'GF-TFW', 'IN', 'LM', 'LM-CF', 'LY', 'NV', 'OC', 'PDS', 'RE', 'SU', 'TR']
day_behaviours_list = [(x,y) for x in list_day_no for y in list_behaviours]


#use for testing 
#test_day_no = range(1, 3)
#test_behaviours = ['SW', 'FO']
#test = [(x,y) for x in test_day_no for y in list_behaviours]


# Functions

In [481]:
def yes_no_every_second(start, finish): #creates an  entry for every second of the day between 5am (18000) and 8pm (72000), 1 for a yes and 0 for a no 
    i = 18000  # ie starts at 5am
    yes_list = []
    if len(start) == 0: #deals with behaviours that do not occur on a given day
        yes_list = ['0']*54000
    while i < 72000: # ends the time span at 8pm and prevents traceback error for behaviours that are not recorded in a given day
        for s in range(0, len(start)): # len(start) is the number of instances of the behaviour recorded that day
            if  i == int(start[s]): # this deals with the start and finish time of a behaviour instance
                yes_list.append(1)
                #print 'yes', i, 'equal to start or finish'
            elif  i == int(finish[s]): # this deals with the start and finish time of a behaviour instance
                yes_list.append(0)
                #print 'no', i, 'equa to finish time which is really start of next behaviour'
            elif i > int(start[s]) and i < int(finish[s]): #deals with seconds in between a start and finish time
                yes_list.append(1)
                #print 'yes', i, 'in between start and finish'
            elif s-1 >= 0 and i > int(finish[s-1]) and i < int(start[s]):
                yes_list.append(0)
                #print 'no', i, 'bigger than finish, less than next start'
            elif s == (len(start) -1) and i > int(finish[s]):
                yes_list.append(0)
                #print 'no', i, 'bigger than last finish but less than 72000'
            elif s == 0 and i < int(start[s]):
                yes_list.append(0)
                #print 'no', i, 'bigger than 18000 but less than 1st start'
                
                
        i = i + 1
    print len(yes_list)
    return yes_list

In [482]:
def tsb(day_behaviour):
    #unpack the tuple
    day, behaviour = day_behaviour
    print day, behaviour
    #subset the data frame
    current = zoo[(zoo.DayNo == day) & (zoo.Behaviour == behaviour)]
    #create lists for start and finish times
    start = current['start_time'].tolist()
    finish = current['finish_time'].tolist()
    #convert into yes no for every second of the day from 5am to 8pm
    yes_no = pd.DataFrame(yes_no_every_second(start, finish), columns = ['yes_no'])
    yes_no['time in sec'] = pd.timedelta_range(start='05:00:00', end='19:59:59', freq='1S')
    yes_no = yes_no.set_index('time in sec')
    #Downsampe into 15 min intervals and convert to % TSB
    TSB = pd.DataFrame((yes_no['yes_no']).resample('15min').sum().apply(lambda x: (float(x)/900)*100))
    TSB = TSB.rename(columns = {'yes_no':'TSB (%)'})
    #add day and behaviour to the data frame    
    TSB['DayNo'] = day
    TSB['Behaviour'] = behaviour
    #print TSB
    return TSB
    
    
    

### Functions for Data Quality checks

In [483]:
def missing_values(df):
    print 'Summary of missing values'
    print zoo.isnull().sum()
    print '###############################'


In [484]:
#checks that every start time is less than the corresponding finish time
#use after times have been converted to seconds
def start_greater_than_finish(df):
    start_greater_than_finish = df.query('start_time > finish_time')
    if len(start_greater_than_finish) == 0:
        print 'Start times are less than corresponding finish times?'
        print 'Pass: All start times less than corresponding finish times'
        print '###############################'
    else:
        print 'Start times are less than corresponding finish times?'
        print 'Fail: some start times greater than corresponding finish times'
        print start_greater_than_finish
        print '###############################'
    
    

In [485]:
#checks that every start time is equal to the previous finish time for a single day
#called within secons function that loops through the days
def start_equals_previous_finish_day(df, day):
       
    current = df[(df.DayNo == day)]
    diff = current.start_time - current.finish_time.shift()
    diff = diff.iloc[1:] #remove first entry as there is no previous finish time for it
    if diff.sum() == 0:
        print 'Day No', day,': Pass'
       
    else:
        print 'Day No', day, 'Fail: The start time does not equal the previous finish time for the following rows'
        print diff[(diff != 0)]        


In [486]:
#loops through all days checking start time is equal to previous finish time
#use after time has been converted to seconds

def start_equals_previous_finish_all(df, list_of_days):
    print 'Every start time equals previous finish time?'
    [start_equals_previous_finish_day(df, day) for day in list_of_days]
    print '###############################'

In [487]:
#check start time is 5am
#use after times converted to seconds
def check_start_5am(df):
    check_start = df.groupby(['DayNo'])['start_time'].min() == 18000
    check_finish = df.groupby(['DayNo'])['finish_time'].max() == 72000
    values = check_start.unique()
    if len(values) == 1 and values[0] == True:
        print 'Start at 5am?'
        print 'Pass: All days start at 5am'
        print '###############################'
    else:
        print 'Start at 5am?'
        print 'Not all days start at 5am'
        print check_start[check_start == False]
        print '###############################'


In [488]:
#check finish time is 8pm
#use after time has been converted to seconds
def check_finish_8pm(df):
    check_finish = zoo.groupby(['DayNo'])['finish_time'].max() == 72000
    values = check_finish.unique()
    if len(values) == 1 and values[0] == True:
        print 'Finish at 8pm?'
        print 'Pass: All days finish at 8pm'
        print '###############################'
    else:
        print 'Finish at 8pm?'
        print 'Fail: Not all days finish at 8pm'
        print check_finish[check_finish == False]
        print '###############################'


In [496]:
#check only behaviours from the desingated list are used
def check_behaviours(df):
    print 'Do the behaviours in the dataset match the allowed list?'
    behaviours_in_data = sorted(df.Behaviour.unique().tolist())
    difference = list(set(behaviours_in_data) - set(list_behaviours))
    if len(difference) == 0:
        print 'Pass: Behaviours in data set and list of allowed behaviours match'
        print '###############################'
    else:
        print 'Fail: There is a difference between behaviours in data set and the allowed list'
        print difference
        print '###############################'



In [497]:
def quality_check(df, list_day_no):
    missing_values(df)
    check_behaviours(df)
    check_start_5am(df)
    check_finish_8pm(df)
    start_equals_previous_finish_all(df, list_day_no)
    start_greater_than_finish(df)


## Import Packages and Set Working Directory

In [491]:
import os
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from scipy import stats
import pylab
os.chdir("C:/Users/bethc/Desktop/DBS/DBS_Project/Data")


## Read in the data and name the columns

In [492]:
#read in required columns from csv file and name columns
col_names =['DayNo', 'Date', 'Behaviour', 'StartTime', 'FinishTime']
zoo = pd.read_csv("master_data_for_pandas.csv", header = None, na_values = 'NaN', usecols=[0, 1,3,4,5], names=col_names, parse_dates = ['Date'])
zoo = zoo.iloc[0:7495] #get rid of trailing cells

## Convert Start and Finish times to seconds

In [493]:
#convert start and finish time to seconds
zoo['start_time'] = pd.to_timedelta(zoo.StartTime)
zoo['start_time'] = zoo['start_time'].dt.total_seconds().astype(int)
zoo['finish_time'] = pd.to_timedelta(zoo.FinishTime)
zoo['finish_time'] = zoo['finish_time'].dt.total_seconds().astype(int)


## Quality Check on Start and Finish times

In [498]:
quality_check(zoo, list_day_no)

Summary of missing values
DayNo          0
Date           0
Behaviour      0
StartTime      0
FinishTime     0
start_time     0
finish_time    0
dtype: int64
###############################
Do the behaviours in the dataset match the allowed list?
Pass: Behaviours in data set and list of allowed behaviours match
###############################
Start at 5am?
Pass: All days start at 5am
###############################
Finish at 8pm?
Pass: All days finish at 8pm
###############################
Every start time equals previous finish time?
Day No 1 : Pass
Day No 2 : Pass
Day No 3 : Pass
Day No 4 : Pass
Day No 5 : Pass
Day No 6 : Pass
Day No 7 : Pass
Day No 8 : Pass
Day No 11 : Pass
Day No 12 : Pass
Day No 13 : Pass
Day No 14 : Pass
Day No 15 : Pass
Day No 16 : Pass
Day No 17 : Pass
Day No 18 : Pass
Day No 19 : Pass
Day No 20 : Pass
Day No 21 : Pass
Day No 22 : Pass
Day No 23 : Pass
Day No 24 : Pass
Day No 25 : Pass
Day No 26 : Pass
Day No 27 : Pass
Day No 28 : Pass
Day No 29 : Pass
Day No 3

In [449]:
all = [tsb(x) for x in day_behaviours_list]
result = pd.concat(all)

1 SW
54000
                TSB (%)  DayNo Behaviour
time in sec                             
05:00:00       0.000000      1        SW
05:15:00       0.000000      1        SW
05:30:00       0.000000      1        SW
05:45:00       0.000000      1        SW
06:00:00       0.000000      1        SW
06:15:00       0.000000      1        SW
06:30:00       0.000000      1        SW
06:45:00       0.000000      1        SW
07:00:00      78.444444      1        SW
07:15:00     100.000000      1        SW
07:30:00      80.888889      1        SW
07:45:00       5.333333      1        SW
08:00:00      11.111111      1        SW
08:15:00      88.222222      1        SW
08:30:00      77.333333      1        SW
08:45:00      43.000000      1        SW
09:00:00      53.333333      1        SW
09:15:00      75.555556      1        SW
09:30:00      17.000000      1        SW
09:45:00       0.000000      1        SW
10:00:00       0.000000      1        SW
10:15:00       0.000000      1        SW
10:30

In [450]:
result.head()

,TSB (%),DayNo,Behaviour
time in sec,,,
05:00:00,0.0,1,SW
05:15:00,0.0,1,SW
05:30:00,0.0,1,SW
05:45:00,0.0,1,SW
06:00:00,0.0,1,SW


In [508]:
#there are 60 time intervals so each day total %TSB should equal 60*100 = 6000
check_6000 = result.groupby(['DayNo'])['TSB (%)'].sum()
check_6000 = check_6000.round(0)
not_6000 = check_6000[(check_6000 != 6000)]
print not_6000

day1 = result[(result.DayNo == 1)]
day1 = day1.groupby(day1.index)['TSB (%)'].sum()
print day1

day5 = result[(result.DayNo == 5)]
day5 = day5.groupby(day5.index)['TSB (%)'].sum()
print day5





DayNo
1    5998.0
5    5999.0
Name: TSB (%), dtype: float64
time in sec
05:00:00    100.000000
05:15:00    100.000000
05:30:00    100.000000
05:45:00    100.000000
06:00:00     98.222222
06:15:00    100.000000
06:30:00    100.000000
06:45:00    100.000000
07:00:00    100.000000
07:15:00    100.000000
07:30:00    100.000000
07:45:00    100.000000
08:00:00    100.000000
08:15:00    100.000000
08:30:00    100.000000
08:45:00    100.000000
09:00:00    100.000000
09:15:00    100.000000
09:30:00    100.000000
09:45:00    100.000000
10:00:00    100.000000
10:15:00    100.000000
10:30:00    100.000000
10:45:00    100.000000
11:00:00    100.000000
11:15:00    100.000000
11:30:00    100.000000
11:45:00    100.000000
12:00:00    100.000000
12:15:00    100.000000
12:30:00    100.000000
12:45:00    100.000000
13:00:00    100.000000
13:15:00    100.000000
13:30:00    100.000000
13:45:00    100.000000
14:00:00    100.000000
14:15:00    100.000000
14:30:00    100.000000
14:45:00    100.000000
15:00:00

In [452]:
#write data to disk
result.to_csv("master_data_after_pandas.csv")

## Calculating the TSS per Day

In [ ]:
#subset just for SW
swaying = result[(result['Behaviour'] == 'SW')]
swaying_mean = swaying.groupby(['DayNo']).mean()
#swaying_mean


## Join with the other metadata

In [ ]:
#read in the metadata
metadata = pd.read_csv("MetaData.csv", na_values = 'NaN', parse_dates = ['Date'])
metadata = metadata.rename(columns = {'Day':'DayNo'})
metadata = metadata.set_index('DayNo')
metadata.head()


In [ ]:
#join on DayNo
joined = metadata.join(swaying_mean, lsuffix='_metadata', rsuffix='_swaying_mean')
joined.to_csv("meta_data_with_tss.csv")

In [ ]:
swaying_mean_by_day_type = joined.groupby(['Day Type']).mean()
swaying_mean_by_day_type

In [ ]:
swaying_median_by_day_type = joined.groupby(['Day Type']).median()
swaying_median_by_day_type

## t-test to compare browse to non-browse days

In [ ]:
#subset browse days and check if assumption of normality holds
browse = joined[joined['Browse'] == 'Yes']['TSB (%)'] # subset for browse days
stats.probplot(browse, dist="norm", plot=pylab)
pylab.show()

from scipy.stats import shapiro
stat, p = shapiro(browse)
print 'Shapiro-Wilk test results for browse: stat', stat, 'p-value', p


In [ ]:
#subset non-browse days and check if assumption of normality holds
no_browse = joined[joined['Browse'] == 'No']['TSB (%)'] # subset for browse days
stats.probplot(no_browse, dist="norm", plot=pylab)
pylab.show()

from scipy.stats import shapiro
stat, p = shapiro(no_browse)
print 'Shapiro-Wilk test results for no browse: stat', stat, 'p-value', p

### test for equal variance

In [ ]:
test_equal_var = scipy.stats.bartlett(browse, no_browse)
test_equal_var

### t-test

In [ ]:
ttest = stats.ttest_ind(browse, no_browse)
statistic, p = ttest
if p > 0.05:
	print 'No difference in means. Fail to reject H0 since p is', p
else:
	print 'Difference in means. Reject H0 since p is', p

In [ ]:
round(browse.mean(), 2)

In [ ]:
round(no_browse.mean(), 2)

In [ ]:
# box plots
#boxplot_browse = joined.boxplot(column = 'TSB (%)', by = ['Day Type'])
#plt.show()

In [ ]:
# box plots
#boxplot_browse = joined.boxplot(column = 'TSB (%)', by = ['DOTW'])
#plt.show()

## One way anova with 4 factors

In [ ]:
normal = joined[joined['Day Type'] == 'Normal']['TSB (%)']
fresh = joined[joined['Day Type'] == 'Fresh']['TSB (%)']
fresh_no21 = fresh.drop([21])
one_day_old = joined[joined['Day Type'] == 'OneDayOld']['TSB (%)']
two_days_old = joined[joined['Day Type'] == 'TwoDaysOld']['TSB (%)']

# Perform the ANOVA
anova = stats.f_oneway(normal, fresh, one_day_old, two_days_old)
#anova

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

tukey = multi.MultiComparison(joined['TSB (%)'], joined['Day Type'])
Results = tukey.tukeyhsd()
print(Results)


## Try with Sunday 3rd (Day 21) removed

In [ ]:
joined_no21 = joined.drop([21])

tukey = multi.MultiComparison(joined_no21['TSB (%)'], joined_no21['Day Type'])
Results = tukey.tukeyhsd()
print(Results)

anova = stats.f_oneway(normal, fresh_no21, one_day_old, two_days_old)
anova




In [ ]:
swaying_no21_mean = joined_no21.groupby(['Day Type']).mean()
swaying_no21_mean

## Include feasability days

In [ ]:
#joined.head()

In [ ]:
#create dataframe for feasability data
feasability = pd.DataFrame({'DayNo': [100, 101], 'Date': ['2017-06-07', '2017-06-08'], 'DOTW': ['Wednesday', 'Thursday'], 'Browse': ['Yes', 'No'], 'Day Type': ['Fresh', 'Normal'], 'Section': ['Feasability', 'Feasability'], 'TSB (%)': [5.5, 15.76]})
feasability = feasability.set_index('DayNo')
feasability.head()

In [ ]:
#add it to the end of the joined data frame
joined_with_feasability = pd.concat([joined, feasability])
joined_with_feasability

#with feasability but Day21 removed
joined_no21_with_feasability = pd.concat([joined_no21, feasability])
joined_no21_with_feasability


In [ ]:
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

tukey_feas = multi.MultiComparison(joined_with_feasability['TSB (%)'], joined_with_feasability['Day Type'])
Results = tukey_feas.tukeyhsd()
print(Results)


In [ ]:
tukey_feas_no21 = multi.MultiComparison(joined_no21_with_feasability['TSB (%)'], joined_no21_with_feasability['Day Type'])
Results = tukey_feas_no21.tukeyhsd()
print(Results)


In [ ]:
def start_equals_previous_finish_test(df):
    i = 1 #start at the 2nd start time
    while i < len(df):
        if start_time.iloc[i] != finish_time.iloc[i-1]:
            start_not_eq_finish.append(start_time)
        i = i + 1
        
    if len(start_eq_previous_finish):
        print 'Pass: All start time equal to previous finish times'
    else:
        print 'Fail: Not all start times are equal to previous finish times'
        print start_eq_previous_finish

In [ ]:
test = pd.DataFrame({'start_time': range(1,10), 'finish_time': range(2,11)})
test

In [ ]:
start_equals_previous_finish_test(test)

In [ ]:
check = test.start_time - test.finish_time.shift()
if check.sum() != 0:
    print 'not all start'
else:
    print 'Pass'




In [ ]:
test.query('start_time != finish_time')

In [171]:
test.finish_time.iloc[-1]

10